In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = './log/style2/128/b0/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 100
test_batch = 100
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/b0/to_star/2000shot/self' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.2
blog_sig = 0.5
jpg_prob = 0.2

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = '_fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'star/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})
student_model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.2, 'drop_connect_rate':0.2})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style2/128/b0/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 4.01M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
# optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_gn0): GroupNorm(8, 32, eps=1e-05, affine=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_gn1): GroupNorm(8, 32, eps=1e-05, affine=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
        (static_padding): Identity()
      )
      (_gn2): GroupNorm(8, 16, ep

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [ ]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += 0.1*sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.001000
Train | 16/16 | Loss:1.2059 | MainLoss:0.9544 | Alpha:0.0261 | SPLoss:6.1044 | CLSLoss:3.2418 | top1:49.6250 | AUROC:0.0000
Test | 123/16 | Loss:0.6935 | MainLoss:0.6935 | SPLoss:6.5323 | CLSLoss:3.1014 | top1:49.9148 | AUROC:0.5021
Test | 32/16 | Loss:0.7147 | MainLoss:0.7147 | SPLoss:6.5323 | CLSLoss:3.1014 | top1:42.8077 | AUROC:0.1481

Epoch: [2 | 1000] LR: 0.001300
Train | 16/16 | Loss:1.6598 | MainLoss:0.6949 | Alpha:0.3333 | SPLoss:0.0937 | CLSLoss:2.8011 | top1:49.3750 | AUROC:0.0000
Test | 123/16 | Loss:0.6934 | MainLoss:0.6934 | SPLoss:0.0695 | CLSLoss:2.4050 | top1:49.9574 | AUROC:0.5061
Test | 32/16 | Loss:0.7250 | MainLoss:0.7250 | SPLoss:0.0695 | CLSLoss:2.4051 | top1:6.5128 | AUROC:0.0136

Epoch: [3 | 1000] LR: 0.001600
Train | 16/16 | Loss:1.3648 | MainLoss:0.6936 | Alpha:0.3332 | SPLoss:0.0164 | CLSLoss:1.9976 | top1:51.2500 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0126 | CLSLoss:1.5797 | top1:49.9476 | AUR

Train | 16/16 | Loss:0.7019 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0256 | CLSLoss:0.0004 | top1:49.0250 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0129 | CLSLoss:0.0004 | top1:50.0000 | AUROC:0.4922
Test | 32/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0129 | CLSLoss:0.0004 | top1:50.0000 | AUROC:0.9693

Epoch: [24 | 1000] LR: 0.003999
Train | 16/16 | Loss:0.7022 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0266 | CLSLoss:0.0005 | top1:50.0000 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0134 | CLSLoss:0.0004 | top1:50.0000 | AUROC:0.5080
Test | 32/16 | Loss:0.6933 | MainLoss:0.6933 | SPLoss:0.0134 | CLSLoss:0.0004 | top1:50.0000 | AUROC:0.0294

Epoch: [25 | 1000] LR: 0.003998
Train | 16/16 | Loss:0.7023 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0269 | CLSLoss:0.0004 | top1:49.8750 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0133 | CLSLoss:0.0004 | top1:50.1114 | AUROC:0.5081
Test | 32/16 | Loss

Train | 16/16 | Loss:0.7012 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0239 | CLSLoss:0.0003 | top1:49.5750 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0106 | CLSLoss:0.0003 | top1:50.0000 | AUROC:0.5080
Test | 32/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0106 | CLSLoss:0.0003 | top1:50.0000 | AUROC:0.0142

Epoch: [46 | 1000] LR: 0.003989
Train | 16/16 | Loss:0.7010 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0231 | CLSLoss:0.0004 | top1:49.0500 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0099 | CLSLoss:0.0005 | top1:50.0000 | AUROC:0.5081
Test | 32/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0099 | CLSLoss:0.0005 | top1:50.0000 | AUROC:0.0166

Epoch: [47 | 1000] LR: 0.003988
Train | 16/16 | Loss:0.7007 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0218 | CLSLoss:0.0004 | top1:49.5500 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0098 | CLSLoss:0.0004 | top1:50.0000 | AUROC:0.4917
Test | 32/16 | Loss

Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:49.3750 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.4914
Test | 32/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.9915

Epoch: [68 | 1000] LR: 0.000397
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:49.6250 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.5087
Test | 32/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.0104

Epoch: [69 | 1000] LR: 0.000397
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:49.8750 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.5091
Test | 32/16 | Loss

Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:49.9750 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.4911
Test | 32/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.9897

Epoch: [90 | 1000] LR: 0.000394
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.1750 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.5086
Test | 32/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.0182

Epoch: [91 | 1000] LR: 0.000394
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.1750 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.5086
Test | 32/16 | Loss

Train | 16/16 | Loss:0.6932 | MainLoss:0.6931 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:49.6000 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.4917
Test | 32/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0000 | top1:50.0000 | AUROC:0.9889

Epoch: [112 | 1000] LR: 0.000390
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:48.9000 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.5092
Test | 32/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.0090

Epoch: [113 | 1000] LR: 0.000390
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:48.9000 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.4919
Test | 32/16 | Lo

Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:49.6500 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.5094
Test | 32/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.0106

Epoch: [134 | 1000] LR: 0.000385
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:49.3750 | AUROC:0.0000
Test | 123/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.4907
Test | 32/16 | Loss:0.6931 | MainLoss:0.6931 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.9887

Epoch: [135 | 1000] LR: 0.000385
Train | 16/16 | Loss:0.6932 | MainLoss:0.6932 | Alpha:0.3333 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.0000
Test | 123/16 | Loss:0.6932 | MainLoss:0.6932 | SPLoss:0.0000 | CLSLoss:0.0001 | top1:50.0000 | AUROC:0.4916
Test | 32/16 | Lo